## Test
asdasdf

In [4]:
import numpy as np 
import matplotlib.pyplot as plt

def pippo():
    plt.plot(np.arange(10), '.-')
    
pippo()

AttributeError: module 'numpy' has no attribute 'plot'

In [3]:

!pip install git+https://github.com/ArcetriAdaptiveOptics/OAO24.git
from oao24 import package_data
dark_image = np.load(package_data.tuto3_folder() / "ID_110.npy")
open_loop_image = np.load(package_data.tuto3_folder() / "ID_105.npy")

  Cloning https://github.com/ArcetriAdaptiveOptics/OAO24.git to /private/var/folders/yy/y6mh4b390sj24kqkcp9nl2c40000gp/T/pip-req-build-s48o78iq
  Running command git clone --filter=blob:none --quiet https://github.com/ArcetriAdaptiveOptics/OAO24.git /private/var/folders/yy/y6mh4b390sj24kqkcp9nl2c40000gp/T/pip-req-build-s48o78iq
  Resolved https://github.com/ArcetriAdaptiveOptics/OAO24.git to commit 73d7ec4d497dbe08e9ead19c51cf2c1e9b56c76b
  Preparing metadata (setup.py) ... done
  Created wheel for oao24: filename=oao24-0.1-py3-none-any.whl size=2503 sha256=23226bb8c1d5f9e20a97dacfdf59ab9c3ccb668139270386870dac2ad94bacbf
  Stored in directory: /private/var/folders/yy/y6mh4b390sj24kqkcp9nl2c40000gp/T/pip-ephem-wheel-cache-v0nemcub/wheels/d7/1c/1a/b24c8e75f080afbcea51969754f3ae16302b3bf76a5b731cb3
Successfully built oao24
  Attempting uninstall: oao24
    Found existing installation: oao24 0.1
    Uninstalling oao24-0.1:
      Successfully uninstalled oao24-0.1


NameError: name 'np' is not defined